<a href="https://colab.research.google.com/github/yutaro-tanaka-yt2705/ag-cgan/blob/main/1_test_gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import gzip
import sys
import warnings
warnings.filterwarnings("ignore")

## Intake data

In [2]:
#read in variant file - it's quicker this way, rather than the pandas read_csv function
filename = '/content/drive/MyDrive/artificial_genome_project/chrX_vcf_formatted.csv.gz'
id_variant_list = []
with gzip.open(filename, "rb") as fi:
  for l in tqdm(fi):
    id_variant_list.append(l.strip().decode("utf-8").split(','))#str(l).replace("b'", '').split(','))
id_variant_df = pd.DataFrame(id_variant_list[1:], columns = id_variant_list[0])
id_variant_df.head()

5097it [00:21, 240.31it/s]


,ID,pair,X_12583_C_T,X_13587_T_C,X_13590_T_C,X_13615_A_G,X_13878_G_C,X_13882_A_C,X_13925_C_A,X_13938_A_G,...,X_156025330_C_T,X_156025332_A_G,X_156025353_C_T,X_156025359_C_T,X_156028989_C_G,X_156029373_T_C,X_156029383_C_G,X_156030556_A_AG,X_156030574_A_AG,X_156030592_A_AG
0,HG00096,A,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HG00096,B,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HG00097,A,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,HG00097,B,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,HG00099,A,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
#delete list variable - save space
del id_variant_list

In [17]:
filtered_variant_info_df = pd.read_csv('/content/drive/MyDrive/artificial_genome_project/chrX_variant_info_filtered_formatted.csv')
filtered_variant_list = filtered_variant_info_df['variant_code'].tolist()

#filter for 1000 variants used to train.
id_variant_df = id_variant_df[['ID', 'pair'] + filtered_variant_list]

In [19]:
id_variant_df.to_csv('/content/drive/MyDrive/artificial_genome_project/chrX_vcf_1000var_formatted.csv', index=False)

---

In [20]:
id_variant_df = pd.read_csv('/content/drive/MyDrive/artificial_genome_project/chrX_vcf_1000var_formatted.csv')
id_variant_df.head()

,ID,pair,X_1343510_CA_C,X_2464340_G_A,X_844565_T_G,X_1443587_C_T,X_846946_A_G,X_155783945_C_T,X_2348441_C_T,X_1495791_C_G,...,X_484011_A_G,X_1494541_A_G,X_1494970_G_A,X_2272724_A_C,X_2441570_T_C,X_1420810_T_C,X_719380_A_G,X_1115457_A_G,X_2449045_T_C,X_1165297_A_G
0,HG00096,A,0,0,0,1,0,1,1,1,...,0,0,0,1,0,0,0,0,0,1
1,HG00096,B,0,0,1,1,1,1,0,1,...,0,0,0,1,0,0,0,1,0,0
2,HG00097,A,0,0,1,1,1,1,0,1,...,0,0,0,0,0,1,0,0,0,0
3,HG00097,B,0,0,1,1,0,1,0,0,...,0,0,0,1,0,0,0,0,1,0
4,HG00099,A,0,0,1,1,1,1,0,0,...,0,0,0,1,1,1,0,0,1,0


In [21]:
#read in target file
target_file = pd.read_csv('/content/drive/MyDrive/artificial_genome_project/people_info_formatted.csv')

#encode population codes, and save as different dataframe.
from sklearn.preprocessing import OrdinalEncoder
ord_enc = OrdinalEncoder()
target_file["population_code"] = ord_enc.fit_transform(target_file[['Population code']])

population_code_df = target_file[['population_code', 'Population code']].drop_duplicates()

#filter out unnecessary columns
target_file = target_file[['Sample name', 'population_code']]
target_file.head()

,Sample name,population_code
0,HG00315,9.0
1,HG00327,9.0
2,HG00334,9.0
3,HG00339,9.0
4,HG00341,9.0


In [22]:
#check population code dataframe
population_code_df.head()

,population_code,Population code
0,9.0,FIN
13,6.0,CHS
15,10.0,GBR
68,22.0,PUR
70,3.0,CDX


In [23]:
#merge target and VCF
id_variant_df = id_variant_df.merge(target_file, left_on='ID', right_on='Sample name')
id_variant_df.head()

,ID,pair,X_1343510_CA_C,X_2464340_G_A,X_844565_T_G,X_1443587_C_T,X_846946_A_G,X_155783945_C_T,X_2348441_C_T,X_1495791_C_G,...,X_1494970_G_A,X_2272724_A_C,X_2441570_T_C,X_1420810_T_C,X_719380_A_G,X_1115457_A_G,X_2449045_T_C,X_1165297_A_G,Sample name,population_code
0,HG00096,A,0,0,0,1,0,1,1,1,...,0,1,0,0,0,0,0,1,HG00096,10.0
1,HG00096,B,0,0,1,1,1,1,0,1,...,0,1,0,0,0,1,0,0,HG00096,10.0
2,HG00097,A,0,0,1,1,1,1,0,1,...,0,0,0,1,0,0,0,0,HG00097,10.0
3,HG00097,B,0,0,1,1,0,1,0,0,...,0,1,0,0,0,0,1,0,HG00097,10.0
4,HG00099,A,0,0,1,1,1,1,0,0,...,0,1,1,1,0,0,1,0,HG00099,10.0


## Test Build GAN


In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import regularizers, layers
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Input, Dense, Activation, LeakyReLU, BatchNormalization
from tensorflow.keras.models import Sequential, load_model, save_model, model_from_json
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
plt.switch_backend('agg')
from sklearn.decomposition import PCA

#Save model
def save_mod(gan, gen, disc, epoch):
    discriminator.trainable = False
    save_model(gan, epoch+"_gan")
    discriminator.trainable = True
    save_model(gen, epoch+"_generator")
    save_model(disc, epoch+"_discriminator")

In [ ]:
latent_size = 600 #size of noise input
alph = 0.01 #alpha value for LeakyReLU
g_learn = 0.0001 #generator learning rate
d_learn = 0.0008 #discriminator learning rate
epochs = 100
batch_size = 32
ag_size = 216 #number of artificial genomes (haplotypes) to be created
gpu_count = 2 #number of GPUs
save_that = 200 #epoch interval for saving outputs
num_classes = len(target_file['population_code'].unique())
num_channels = 1

In [ ]:
#get variants only from id_variant_df
X = id_variant_df.iloc[:, 2:-2]
#get target list from id_variant_df
y = id_variant_df.iloc[:, -1]
#make labels into categorical binary class matrix
all_labels = keras.utils.to_categorical(y, num_classes)

In [ ]:
#Make generator
generator = Sequential()
generator.add(Dense(int(X.shape[1]//1.2), input_shape=(latent_size,), kernel_regularizer=regularizers.l2(0.0001)))
generator.add(LeakyReLU(alpha=alph))
generator.add(Dense(int(X.shape[1]//1.1), kernel_regularizer=regularizers.l2(0.0001)))
generator.add(LeakyReLU(alpha=alph))
generator.add(Dense(X.shape[1], activation = 'tanh'))

#Make discriminator
discriminator = Sequential()
discriminator.add(Dense(X.shape[1]//2, input_shape=(X.shape[1],), kernel_regularizer=regularizers.l2(0.0001)))
discriminator.add(LeakyReLU(alpha=alph))
discriminator.add(Dense(X.shape[1]//3, kernel_regularizer=regularizers.l2(0.0001)))
discriminator.add(LeakyReLU(alpha=alph))
discriminator.add(Dense(1, activation = 'sigmoid'))
#if gpu_count > 1:
#    discriminator = multi_gpu_model(discriminator, gpus=gpu_count)
discriminator.compile(optimizer=Adam(learning_rate=d_learn), loss='binary_crossentropy')

#Set discriminator to non-trainable
discriminator.trainable = False

In [ ]:
#Make GAN
gan = Sequential()
gan.add(generator)
gan.add(discriminator)
#if gpu_count > 1:
#    gan = multi_gpu_model(gan, gpus=gpu_count)
gan.compile(optimizer=Adam(learning_rate=g_learn), loss='binary_crossentropy')

y_real, y_fake = np.ones([batch_size, 1]), np.zeros([batch_size, 1])
X_real = df_noname.values

losses = []
batches = len(X_real)//batch_size

In [ ]:
#clear backend session.
K.clear_session()

In [ ]:
X.head()

,X_12583_C_T,X_13587_T_C,X_13590_T_C,X_13615_A_G,X_13878_G_C,X_13882_A_C,X_13925_C_A,X_13938_A_G,X_13953_C_T,X_14612_C_G,...,X_156025330_C_T,X_156025332_A_G,X_156025353_C_T,X_156025359_C_T,X_156028989_C_G,X_156029373_T_C,X_156029383_C_G,X_156030556_A_AG,X_156030574_A_AG,X_156030592_A_AG
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
y.head()

0    10.0
1    10.0
2    10.0
3    10.0
4    10.0
Name: population_code, dtype: float64

In [ ]:
X_reshape = np.reshape(X, (-1, 106962, 1, 1))
X_reshape.shape

ValueError: ignored

In [ ]:
#make into tensorflow dataset
dataset = tf.data.Dataset.from_tensor_slices((X, all_labels))
dataset = dataset.shuffle(buffer_size=1024).batch(batch_size)

In [ ]:
#set num of in-channels
generator_in_channels = latent_size + num_classes
discriminator_in_channels = num_channels + num_classes
print(generator_in_channels, discriminator_in_channels)

626 27


In [ ]:
# Create the discriminator.
discriminator = keras.Sequential(
    [
        keras.layers.InputLayer((28, 28, discriminator_in_channels)),
        layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.GlobalMaxPooling2D(),
        layers.Dense(1),
    ],
    name="discriminator",
)

In [ ]:
# Create the generator.
generator = keras.Sequential(
    [
        keras.layers.InputLayer((generator_in_channels,)),
        # We want to generate 128 + num_classes coefficients to reshape into a
        # 7x7x(128 + num_classes) map.
        layers.Dense(7 * 7 * generator_in_channels),
        layers.LeakyReLU(alpha=0.2),
        layers.Reshape((7, 7, generator_in_channels)),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(1, (7, 7), padding="same", activation="sigmoid"),
    ],
    name="generator",
)

In [ ]:
class ConditionalGAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super(ConditionalGAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim
        self.gen_loss_tracker = keras.metrics.Mean(name="generator_loss")
        self.disc_loss_tracker = keras.metrics.Mean(name="discriminator_loss")

    @property
    def metrics(self):
        return [self.gen_loss_tracker, self.disc_loss_tracker]

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super(ConditionalGAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn

    def train_step(self, data):
        # Unpack the data.
        real_images, one_hot_labels = data

        # Add dummy dimensions to the labels so that they can be concatenated with
        # the images. This is for the discriminator.
        image_one_hot_labels = one_hot_labels[:, :, None, None]
        image_one_hot_labels = tf.repeat(
            image_one_hot_labels, repeats=[image_size * image_size]
        )
        image_one_hot_labels = tf.reshape(
            image_one_hot_labels, (-1, image_size, image_size, num_classes)
        )

        # Sample random points in the latent space and concatenate the labels.
        # This is for the generator.
        batch_size = tf.shape(real_images)[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        random_vector_labels = tf.concat(
            [random_latent_vectors, one_hot_labels], axis=1
        )

        # Decode the noise (guided by labels) to fake images.
        generated_images = self.generator(random_vector_labels)

        # Combine them with real images. Note that we are concatenating the labels
        # with these images here.
        fake_image_and_labels = tf.concat([generated_images, image_one_hot_labels], -1)
        real_image_and_labels = tf.concat([real_images, image_one_hot_labels], -1)
        combined_images = tf.concat(
            [fake_image_and_labels, real_image_and_labels], axis=0
        )

        # Assemble labels discriminating real from fake images.
        labels = tf.concat(
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0
        )

        # Train the discriminator.
        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(
            zip(grads, self.discriminator.trainable_weights)
        )

        # Sample random points in the latent space.
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        random_vector_labels = tf.concat(
            [random_latent_vectors, one_hot_labels], axis=1
        )

        # Assemble labels that say "all real images".
        misleading_labels = tf.zeros((batch_size, 1))

        # Train the generator (note that we should *not* update the weights
        # of the discriminator)!
        with tf.GradientTape() as tape:
            fake_images = self.generator(random_vector_labels)
            fake_image_and_labels = tf.concat([fake_images, image_one_hot_labels], -1)
            predictions = self.discriminator(fake_image_and_labels)
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        # Monitor loss.
        self.gen_loss_tracker.update_state(g_loss)
        self.disc_loss_tracker.update_state(d_loss)
        return {
            "g_loss": self.gen_loss_tracker.result(),
            "d_loss": self.disc_loss_tracker.result(),
        }

In [ ]:
cond_gan = ConditionalGAN(
    discriminator=discriminator, generator=generator, latent_dim=latent_size
)
cond_gan.compile(
    d_optimizer=keras.optimizers.Adam(learning_rate=0.0003),
    g_optimizer=keras.optimizers.Adam(learning_rate=0.0003),
    loss_fn=keras.losses.BinaryCrossentropy(from_logits=True),
)

cond_gan.fit(dataset, epochs=20)

In [ ]:
#Make generator
generator = Sequential()
generator.add(Dense(int(X.shape[1]//1.2), input_shape=(latent_size,), kernel_regularizer=regularizers.l2(0.0001)))
generator.add(LeakyReLU(alpha=alph))
generator.add(Dense(int(X.shape[1]//1.1), kernel_regularizer=regularizers.l2(0.0001)))
generator.add(LeakyReLU(alpha=alph))
generator.add(Dense(X.shape[1], activation = 'tanh'))

#Make discriminator
discriminator = Sequential()
discriminator.add(Dense(X.shape[1]//2, input_shape=(X.shape[1],), kernel_regularizer=regularizers.l2(0.0001)))
discriminator.add(LeakyReLU(alpha=alph))
discriminator.add(Dense(X.shape[1]//3, kernel_regularizer=regularizers.l2(0.0001)))
discriminator.add(LeakyReLU(alpha=alph))
discriminator.add(Dense(1, activation = 'sigmoid'))
#if gpu_count > 1:
#    discriminator = multi_gpu_model(discriminator, gpus=gpu_count)
discriminator.compile(optimizer=Adam(learning_rate=d_learn), loss='binary_crossentropy')

#Set discriminator to non-trainable
discriminator.trainable = False